In [130]:
import pandas as pd
from collections import namedtuple

input_file="input.txt"

with open(input_file) as file:
    lines = file.readlines()
    lines = map(lambda x: list(x.strip()), lines)
    df = pd.DataFrame(lines)
    
MovementIndicator = namedtuple('MovementIndicator', ['horizontal', 'vertical'])
Position = namedtuple('Position', ['x', 'y'])


def turn_right(movement_indicator: MovementIndicator):
    top = MovementIndicator(vertical=-1, horizontal=0)
    down = MovementIndicator(vertical=1, horizontal=0)
    right = MovementIndicator(vertical=0, horizontal=1)
    left = MovementIndicator(vertical=0, horizontal=-1)
    
    
    #right_to_down
    if movement_indicator == right:
        return MovementIndicator(horizontal=0, vertical=1)
    #down_to_left
    if movement_indicator == down:
        return left
    #left_to_top
    if movement_indicator == left:
        return top
    #top_to_right
    if movement_indicator == top:
        return right
    
    
def get_next_position(position: Position, movement_indicator: MovementIndicator):
    return Position(x = position.x+movement_indicator.horizontal, y = position.y + movement_indicator.vertical)
   
df

,0,1,2,3,4,5,6,7,8,9,...,120,121,122,123,124,125,126,127,128,129
0,.,.,.,.,.,.,.,.,#,.,...,.,.,.,.,.,.,.,.,.,.
1,.,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,#,.,#,.
2,.,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.
3,.,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.
4,.,.,.,.,.,.,.,.,.,#,...,.,.,.,.,.,.,.,.,.,#
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,.,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,#,.
126,.,.,.,#,.,.,.,.,#,.,...,.,.,.,.,.,.,.,.,.,.
127,.,.,.,.,#,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.
128,.,#,.,.,.,#,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.


In [131]:
location = [Position(y =i, x= row[row == "^"].index[0]) for i, row in df.iterrows() if "^" in row.values]
location

[Position(x=np.int64(66), y=90)]

In [132]:

position = location[0]
movement_indicator = MovementIndicator(vertical=-1, horizontal=0)
visited_positions = {position: True}
rows, columns = df.shape

while(position.x >= 0 and position.y >= 0 and position.x<columns and position.y<columns):
    next_position = get_next_position(position, movement_indicator)
    if(next_position.x < 0 or next_position.y < 0 or next_position.x>=columns or next_position.y>=rows):
        break
    next_field_content = df.iloc[next_position.y, next_position.x]
    if next_field_content == ".":
        #move
        visited_positions[next_position]=True
        df.iloc[position.y, position.x] = "."
        position = next_position
    elif next_field_content == "#":
        movement_indicator = turn_right(movement_indicator)
        
len(set(visited_positions))


5086

# Part 2

In [133]:
def detect_circle(start_position, obstacle: Position, df):
    if df.iloc[obstacle.y, obstacle.x] == "^" or df.iloc[obstacle.y, obstacle.x] == "#":
        return False
    field = df.copy(deep=True) 
    field.iloc[obstacle.y, obstacle.x]="#"
    
    position = start_position
    movement_indicator = MovementIndicator(vertical=-1, horizontal=0)
    visited_positions = {(position, movement_indicator): True}
    rows, columns = field.shape
    while(position.x >= 0 and position.y >= 0 and position.x<columns and position.y<columns):
        next_position = get_next_position(position, movement_indicator)
        if(next_position.x < 0 or next_position.y < 0 or next_position.x>=columns or next_position.y>=rows):
            return False
        next_field_content = field.iloc[next_position.y, next_position.x]
        if next_field_content == ".":
            #move
            field.iloc[position.y, position.x] = "."
            position = next_position
            if (position, movement_indicator) in visited_positions: return True
            visited_positions[(position, movement_indicator)]=True
        elif next_field_content == "#":
            movement_indicator = turn_right(movement_indicator)
            if (position, movement_indicator) in visited_positions: return True
   


def is_valid_position(position, shape):
    rows, columns = shape
    return position.x >= 0 and position.y >= 0 and position.x<columns and position.y<columns

potential_obstacles = visited_positions

In [134]:
circles = 0
for index, potential_obstacle in enumerate(potential_obstacles):
    # if(index % 10 == 0): print("Index: " + str(index))
    if detect_circle(start_position=location[0], obstacle=potential_obstacle, df=df):
        circles+=1

KeyboardInterrupt: 

In [ ]:
circles

1770